In [34]:
import pandas as pd
import tensorflow as tf
import tensorflow_decision_forests as tfdf
from sklearn.model_selection import train_test_split
import numpy as np



LOADING DATA

In [35]:
pep_data = pd.read_csv("train_peptides.csv")
prot_data = pd.read_csv("train_proteins.csv")
clin_data = pd.read_csv("train_clinical_data.csv")

In [36]:
pep_data.head()

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7


In [37]:
prot_data.head()

,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0


In [38]:
clin_data.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On


In [39]:
prot_data.shape


(232741, 5)

In [40]:
prot_data["visit_month"].unique()

array([  0,   3,   6,  12,  18,  24,  30,  36,  48,  54,  60,  72,  84,
        96, 108])

Grouping the data

In [41]:
prot_group = prot_data.groupby(["visit_id", "UniProt"])["NPX"].mean().reset_index()

In [42]:
pep_group = pep_data.groupby(["visit_id", "Peptide"])["PeptideAbundance"].mean().reset_index()

Pivot and Merge DF's

In [43]:
prot_trian = prot_group.pivot(index="visit_id", columns="UniProt", values="NPX").rename_axis(columns=None).reset_index()

In [44]:
pep_train = pep_group.pivot(index="visit_id", columns="Peptide", values="PeptideAbundance").rename_axis(columns=None).reset_index()

In [45]:
prot_pep_train = prot_trian.merge(pep_train, how= "left", on=["visit_id"])
prot_pep_train

,visit_id,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,10053_0,9104.27,402321.0,NaN,NaN,7150.57,2497.84,83002.9,15113.6,167327.0,...,202274.0,NaN,4401830.0,77482.6,583075.0,76705.7,104260.0,530223.0,NaN,7207.30
1,10053_12,10464.20,435586.0,NaN,NaN,NaN,NaN,197117.0,15099.1,164268.0,...,201009.0,NaN,5001750.0,36745.3,355643.0,92078.1,123254.0,453883.0,49281.9,25332.80
2,10053_18,13235.70,507386.0,7126.96,24525.7,NaN,2372.71,126506.0,16289.6,168107.0,...,220728.0,NaN,5424380.0,39016.0,496021.0,63203.6,128336.0,447505.0,52389.1,21235.70
3,10138_12,12600.20,494581.0,9165.06,27193.5,22506.10,6015.90,156313.0,54546.4,204013.0,...,188362.0,9433.71,3900280.0,48210.3,328482.0,89822.1,129964.0,552232.0,65657.8,9876.98
4,10138_24,12003.20,522138.0,4498.51,17189.8,29112.40,2665.15,151169.0,52338.1,240892.0,...,206187.0,6365.15,3521800.0,69984.6,496737.0,80919.3,111799.0,NaN,56977.6,4903.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,8699_24,9983.00,400290.0,24240.10,NaN,16943.50,6303.17,77493.6,46435.3,254247.0,...,289888.0,8615.27,8770410.0,33599.1,926094.0,118897.0,133682.0,571879.0,80268.3,54889.70
1109,942_12,6757.32,360858.0,18367.60,14760.7,18603.40,1722.77,86847.4,37741.3,212132.0,...,173259.0,4767.63,374307.0,35767.3,250397.0,65966.9,77976.8,486239.0,45032.7,NaN
1110,942_24,NaN,352722.0,22834.90,23393.1,16693.50,1487.91,114772.0,36095.7,185836.0,...,185428.0,5554.53,NaN,64049.8,479473.0,68505.7,74483.1,561398.0,52916.4,21847.60
1111,942_48,11627.80,251820.0,22046.50,26360.5,22440.20,2117.43,82241.9,30146.6,167633.0,...,137611.0,6310.09,NaN,28008.8,231359.0,63265.8,64601.8,632782.0,51123.7,20700.30


In [46]:
prot_pep_train.shape

(1113, 1196)

In [47]:
df = prot_pep_train.merge(clin_data[["visit_id", "patient_id", "visit_month", "updrs_1"]], how= "left", on="visit_id")
df


,visit_id,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,...,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK,patient_id,visit_month,updrs_1
0,10053_0,9104.27,402321.0,NaN,NaN,7150.57,2497.84,83002.9,15113.6,167327.0,...,77482.6,583075.0,76705.7,104260.0,530223.0,NaN,7207.30,10053.0,0.0,3.0
1,10053_12,10464.20,435586.0,NaN,NaN,NaN,NaN,197117.0,15099.1,164268.0,...,36745.3,355643.0,92078.1,123254.0,453883.0,49281.9,25332.80,10053.0,12.0,4.0
2,10053_18,13235.70,507386.0,7126.96,24525.7,NaN,2372.71,126506.0,16289.6,168107.0,...,39016.0,496021.0,63203.6,128336.0,447505.0,52389.1,21235.70,10053.0,18.0,2.0
3,10138_12,12600.20,494581.0,9165.06,27193.5,22506.10,6015.90,156313.0,54546.4,204013.0,...,48210.3,328482.0,89822.1,129964.0,552232.0,65657.8,9876.98,10138.0,12.0,3.0
4,10138_24,12003.20,522138.0,4498.51,17189.8,29112.40,2665.15,151169.0,52338.1,240892.0,...,69984.6,496737.0,80919.3,111799.0,NaN,56977.6,4903.09,10138.0,24.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,8699_24,9983.00,400290.0,24240.10,NaN,16943.50,6303.17,77493.6,46435.3,254247.0,...,33599.1,926094.0,118897.0,133682.0,571879.0,80268.3,54889.70,8699.0,24.0,11.0
1109,942_12,6757.32,360858.0,18367.60,14760.7,18603.40,1722.77,86847.4,37741.3,212132.0,...,35767.3,250397.0,65966.9,77976.8,486239.0,45032.7,NaN,942.0,12.0,5.0
1110,942_24,NaN,352722.0,22834.90,23393.1,16693.50,1487.91,114772.0,36095.7,185836.0,...,64049.8,479473.0,68505.7,74483.1,561398.0,52916.4,21847.60,942.0,24.0,2.0
1111,942_48,11627.80,251820.0,22046.50,26360.5,22440.20,2117.43,82241.9,30146.6,167633.0,...,28008.8,231359.0,63265.8,64601.8,632782.0,51123.7,20700.30,942.0,48.0,2.0


In [48]:
merged_df = df.dropna(subset=["updrs_1"])

In [49]:
merged_df.columns

Index(['visit_id', 'O00391', 'O00533', 'O00584', 'O14498', 'O14773', 'O14791',
       'O15240', 'O15394', 'O43505',
       ...
       'YVM(UniMod_35)LPVADQDQC(UniMod_4)IR', 'YVMLPVADQDQC(UniMod_4)IR',
       'YVNKEIQNAVNGVK', 'YWGVASFLQK', 'YYC(UniMod_4)FQGNQFLR', 'YYTYLIMNK',
       'YYWGGQYTWDMAK', 'patient_id', 'visit_month', 'updrs_1'],
      dtype='object', length=1199)

In [50]:
merged_df.drop(columns="visit_id")

,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,O60888,...,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK,patient_id,visit_month,updrs_1
0,9104.27,402321.0,NaN,NaN,7150.57,2497.84,83002.9,15113.6,167327.0,129048.0,...,77482.6,583075.0,76705.7,104260.0,530223.0,NaN,7207.30,10053.0,0.0,3.0
1,10464.20,435586.0,NaN,NaN,NaN,NaN,197117.0,15099.1,164268.0,108114.0,...,36745.3,355643.0,92078.1,123254.0,453883.0,49281.9,25332.80,10053.0,12.0,4.0
2,13235.70,507386.0,7126.96,24525.7,NaN,2372.71,126506.0,16289.6,168107.0,163776.0,...,39016.0,496021.0,63203.6,128336.0,447505.0,52389.1,21235.70,10053.0,18.0,2.0
3,12600.20,494581.0,9165.06,27193.5,22506.10,6015.90,156313.0,54546.4,204013.0,56725.0,...,48210.3,328482.0,89822.1,129964.0,552232.0,65657.8,9876.98,10138.0,12.0,3.0
4,12003.20,522138.0,4498.51,17189.8,29112.40,2665.15,151169.0,52338.1,240892.0,85767.1,...,69984.6,496737.0,80919.3,111799.0,NaN,56977.6,4903.09,10138.0,24.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,9983.00,400290.0,24240.10,NaN,16943.50,6303.17,77493.6,46435.3,254247.0,138910.0,...,33599.1,926094.0,118897.0,133682.0,571879.0,80268.3,54889.70,8699.0,24.0,11.0
1109,6757.32,360858.0,18367.60,14760.7,18603.40,1722.77,86847.4,37741.3,212132.0,100519.0,...,35767.3,250397.0,65966.9,77976.8,486239.0,45032.7,NaN,942.0,12.0,5.0
1110,NaN,352722.0,22834.90,23393.1,16693.50,1487.91,114772.0,36095.7,185836.0,99183.5,...,64049.8,479473.0,68505.7,74483.1,561398.0,52916.4,21847.60,942.0,24.0,2.0
1111,11627.80,251820.0,22046.50,26360.5,22440.20,2117.43,82241.9,30146.6,167633.0,84875.1,...,28008.8,231359.0,63265.8,64601.8,632782.0,51123.7,20700.30,942.0,48.0,2.0


SPLIT DATA
split data into train en test set and check shapes to confirm 

In [51]:
X = merged_df

In [52]:
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

In [53]:
X_train.shape

(854, 1199)

In [54]:
X_test.shape

(214, 1199)

In [55]:
X.shape

(1068, 1199)

CHANGE FROM DF TO DATASET
tensorflow model needs tf dataset instead of pd dataframe

In [56]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(X_train, label="updrs_1", task=tfdf.keras.Task.REGRESSION)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(X_test, label="updrs_1", task=tfdf.keras.Task.REGRESSION)


TRAINING MODEL

In [57]:
rf = tfdf.keras.RandomForestModel(task=tfdf.keras.Task.REGRESSION, verbose=0)
rf.compile(metrics=["mse"])

In [58]:
rf.fit(x=train_ds)

[INFO 24-01-23 13:28:52.4748 CET kernel.cc:1233] Loading model from path /var/folders/ws/dtdg5kqn1dx9rls1zt3mr4c80000gn/T/tmplklybvok/model/ with prefix c8443ab44cd147aa
[INFO 24-01-23 13:28:52.6642 CET decision_forest.cc:660] Model loaded with 300 root(s), 70972 node(s), and 1197 input feature(s).
[INFO 24-01-23 13:28:52.6643 CET abstract_model.cc:1344] Engine "RandomForestOptPred" built
[INFO 24-01-23 13:28:52.6647 CET kernel.cc:1061] Use fast generic engine


EVALUATE MODEL

In [59]:
inspector = rf.make_inspector()
inspector.evaluation()
evaluation = rf.evaluate(x=test_ds)

1/1 [==============================] - 3s 3s/step - loss: 0.0000e+00 - mse: 19.5633


sMAPE: kaggles metric for the models success

In [60]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [65]:
preds = rf.predict(test_ds)
smape(X_test["updrs_1"].values.tolist(), preds.flatten())



1/1 [==============================] - 1s 1s/step


71.23668286331902